In [ ]:
pip install nibabel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 28.0 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def build_mobilenetv3_segmentation(input_shape):
    base_model = tf.keras.applications.MobileNetV3Large(input_shape=input_shape,
                                                        include_top=False,
                                                        weights='imagenet')

    # Add segmentation head (Upsampling layers for segmentation task)
    x = base_model.output
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D(2)(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D(2)(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D(2)(x)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D(2)(x)
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D(2)(x)

    # Final segmentation layer (binary classification)
    output = layers.Conv2D(1, (1, 1), activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=output)

    # Compile the model for segmentation task
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

input_shape = (256, 256, 3)  # Modify this based on dataset input dimensions
model = build_mobilenetv3_segmentation(input_shape)


12683000/12683000 [==============================] - 0s 0us/step


In [ ]:
!kaggle datasets download -d felipemeganha/chest-xray-masks-and-labels-images
!unzip chest-xray-masks-and-labels-images.zip -d chest_xray_dataset

Dataset URL: https://www.kaggle.com/datasets/felipemeganha/chest-xray-masks-and-labels-images
License(s): CC0-1.0
100% 4.07G/4.09G [00:31<00:00, 99.0MB/s]
100% 4.09G/4.09G [00:31<00:00, 140MB/s] 
Archive:  chest-xray-masks-and-labels-images.zip
  inflating: chest_xray_dataset/CXR_png/CHNCXR_0002_0.png  
  inflating: chest_xray_dataset/CXR_png/CHNCXR_0003_0.png  
  inflating: chest_xray_dataset/CXR_png/CHNCXR_0004_0.png  
  inflating: chest_xray_dataset/CXR_png/CHNCXR_0005_0.png  
  inflating: chest_xray_dataset/CXR_png/CHNCXR_0006_0.png  
  inflating: chest_xray_dataset/CXR_png/CHNCXR_0007_0.png  
  inflating: chest_xray_dataset/CXR_png/CHNCXR_0008_0.png  
  inflating: chest_xray_dataset/CXR_png/CHNCXR_0009_0.png  
  inflating: chest_xray_dataset/CXR_png/CHNCXR_0010_0.png  
  inflating: chest_xray_dataset/CXR_png/CHNCXR_0011_0.png  
  inflating: chest_xray_dataset/CXR_png/CHNCXR_0012_0.png  
  inflating: chest_xray_dataset/CXR_png/CHNCXR_0013_0.png  
  inflating: chest_xray_dataset/CXR

In [ ]:
import os
import shutil

# Set paths
chest_xray_dir = 'chest_xray_dataset'
image_dir = os.path.join(chest_xray_dir, 'CXR_png')
mask_dir = os.path.join(chest_xray_dir, 'masks')

# Create directories
os.makedirs(image_dir, exist_ok=True)
os.makedirs(mask_dir, exist_ok=True)

# Move image files and mask files to separate directories
for file in os.listdir(chest_xray_dir):
    if 'mask' in file:
        shutil.move(os.path.join(chest_xray_dir, file), mask_dir)
    elif file.endswith('.png') or file.endswith('.jpg'):
        shutil.move(os.path.join(chest_xray_dir, file), image_dir)


In [ ]:
import cv2
import os
import numpy as np

def load_chest_xray_dataset(image_dir, mask_dir, img_size=(256, 256)):
    images, masks = [], []
    for image_file, mask_file in (zip(os.listdir(image_dir),os.listdir(mask_dir))):
        img_path = os.path.join(image_dir, image_file)
        mask_path = os.path.join(mask_dir, mask_file)

        # Load image and mask
        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Check if the image and mask were loaded successfully
        if img is None:
            print(f"Warning: Could not load image: {img_path}")
            continue
        if mask is None:
            print(f"Warning: Could not load mask: {mask_path}")
            continue

        # Resize and normalize
        img = cv2.resize(img, img_size) / 255.0
        mask = cv2.resize(mask, img_size) / 255.0

        images.append(img)
        masks.append(np.expand_dims(mask, axis=-1))  # Add channel dimension

    return np.array(images), np.array(masks)

xray_images, xray_masks = load_chest_xray_dataset(image_dir, mask_dir)

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
image_dir = '/content/drive/MyDrive/dataset/ProstateX_dataset'
mask_dir = '/content/drive/MyDrive/dataset/ProstateX_mask/ProstateX_mask/images'

# Function to load images and masks
def load_data(image_dir, mask_dir):
    image_files = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])
    mask_files = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])

    images = []
    masks = []

    for img_path, mask_path in zip(image_files, mask_files):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Check if both image and mask are loaded correctly
        if img is None or mask is None:
            print(f"Error loading image or mask: {img_path}, {mask_path}")
            continue

        # Resize images and masks to a consistent size (if needed)
        img = cv2.resize(img, (256, 256))  # Adjust size as necessary
        mask = cv2.resize(mask, (16, 16))

        images.append(img)
        masks.append(mask)

    images = np.array(images) / 255.0  # Normalize images
    masks = np.array(masks) / 255.0    # Normalize masks

    return np.array(images), np.array(masks)

# Load the data
images, masks = load_data(image_dir, mask_dir)


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Function to preprocess images and masks (resize and normalize)
def preprocess_data(images, masks, img_size=(256, 256)):
    processed_images = [cv2.resize(img, img_size) / 255.0 for img in images]
    processed_masks = [cv2.resize(mask, img_size, interpolation=cv2.INTER_NEAREST) / 255.0 for mask in masks]
    return np.array(processed_images), np.array(processed_masks)

# Load and preprocess each dataset
# Example: preprocess chest X-ray dataset
xray_images, xray_masks = preprocess_data(xray_images, xray_masks)

In [ ]:
from sklearn.model_selection import train_test_split
# Training the model sequentially on each dataset
def train_on_dataset(model, images, masks, epochs=5, batch_size=16):
     # Split the data manually into train and validation sets
    train_images, val_images, train_masks, val_masks = train_test_split(images, masks, test_size=0.2, random_state=42)

    model.fit(train_images, train_masks, epochs=epochs, batch_size=batch_size, validation_data=(val_images, val_masks))
train_on_dataset(model, xray_images, xray_masks)
model.save_weights('mobilenetv3_chest_xray.weights.h5')

Epoch 1/5
36/36 [==============================] - 42s 776ms/step - loss: 0.1106 - accuracy: 0.7192 - val_loss: 0.0652 - val_accuracy: 0.7401
Epoch 2/5
36/36 [==============================] - 25s 694ms/step - loss: 0.0154 - accuracy: 0.7476 - val_loss: 0.0500 - val_accuracy: 0.7401
Epoch 3/5
36/36 [==============================] - 25s 696ms/step - loss: 0.0153 - accuracy: 0.7476 - val_loss: 0.0488 - val_accuracy: 0.7401
Epoch 4/5
36/36 [==============================] - 24s 680ms/step - loss: 0.0147 - accuracy: 0.7476 - val_loss: 0.0627 - val_accuracy: 0.7401
Epoch 5/5
36/36 [==============================] - 24s 659ms/step - loss: 0.0129 - accuracy: 0.7476 - val_loss: 0.0623 - val_accuracy: 0.7401


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Function to preprocess images and masks (resize and normalize)
def preprocess_data(images, masks, img_size=(256, 256)):
    processed_images = [cv2.resize(img, img_size) / 255.0 for img in images]
    processed_masks = [cv2.resize(mask, img_size, interpolation=cv2.INTER_NEAREST) / 255.0 for mask in masks]
    return np.array(processed_images), np.array(processed_masks)

# Load and preprocess each dataset
# Example: preprocess chest X-ray dataset
prostratex_images, prostratex_masks = preprocess_data(images,masks)

In [ ]:
from sklearn.model_selection import train_test_split
# Training the model sequentially on each dataset
def train_on_dataset(model, images, masks, epochs=5, batch_size=16):
     # Split the data manually into train and validation sets
    train_images, val_images, train_masks, val_masks = train_test_split(images, masks, test_size=0.2, random_state=42)

    model.fit(train_images, train_masks, epochs=epochs, batch_size=batch_size, validation_data=(val_images, val_masks))
model.load_weights('mobilenetv3_chest_xray.weights.h5')
train_on_dataset(model, prostratex_images, prostratex_masks)
model.save_weights('mobilenetv3_prostratex.weights.h5')

Epoch 1/5
101/101 [==============================] - 68s 678ms/step - loss: 4.8410e-04 - accuracy: 0.9900 - val_loss: 0.0317 - val_accuracy: 0.9900
Epoch 2/5
101/101 [==============================] - 69s 686ms/step - loss: 3.5307e-04 - accuracy: 0.9900 - val_loss: 0.0129 - val_accuracy: 0.9900
Epoch 3/5
101/101 [==============================] - 69s 679ms/step - loss: 3.3343e-04 - accuracy: 0.9900 - val_loss: 0.0049 - val_accuracy: 0.9900
Epoch 4/5
101/101 [==============================] - 69s 681ms/step - loss: 3.2488e-04 - accuracy: 0.9900 - val_loss: 0.0033 - val_accuracy: 0.9900
Epoch 5/5
101/101 [==============================] - 69s 679ms/step - loss: 3.2031e-04 - accuracy: 0.9900 - val_loss: 0.0026 - val_accuracy: 0.9900
